In [1]:
import numpy as np 
import pandas as pd 
import os 
import matplotlib.pyplot as plt
current_dir = 'f:/buaa/python/final_proj'
os.chdir(current_dir)
sensor_file = 'data/mysenior_sensor_data_output.csv'
debris_file = 'data/0_ALEXIS_22638_debris_new.csv'
star_file = 'data/Star-100027_star.csv'
star_magnitude_file = 'data/Star-100027_magnitude.csv'
access_file_csv = 'data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.csv'
access_file_txt = 'data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.txt'

In [2]:
def read_access_data(access_file_csv,access_file_txt,skiprows=29):
  access = pd.read_csv(access_file_csv)
  access_df = pd.read_csv(access_file_txt,skiprows=skiprows,header=None)
  access_ls = []
  access = access.drop(list(range(1,len(access),2)))
  access.reset_index(drop=True,inplace=True)

  for i in range(len(access_df)):
    if access_df.iloc[i,0].replace(' ','').startswith('Access'):
      access_ls.append(access_df.iloc[i-2,0].split('-',maxsplit=2)[-1])
  access['name']=access_ls
  access.set_index('name',inplace=True)
  access['Start Time (UTCG)'] = pd.to_datetime(access['Start Time (UTCG)'])
  access['Stop Time (UTCG)'] = pd.to_datetime(access['Stop Time (UTCG)'])
  return access

access = read_access_data(access_file_csv,access_file_txt,29)

In [4]:
access_names = access.index.tolist()
print(access_names)

['0_OPS_5721_09415', '0_SSU_2_08836', '0_USA_22_17997', 'ASTRID_1_23464', 'COBE_20322', 'COSMOS_1087_11302', 'COSMOS_1176_11788', 'COSMOS_1181_11803', 'COSMOS_1302_12791', 'COSMOS_1357_13160', 'COSMOS_1430_13762', 'COSMOS_1431_13763', 'COSMOS_1476_14174', 'COSMOS_1637_15619', 'COSMOS_1831_17586', 'COSMOS_1927_18940', 'COSMOS_1930_18943', 'COSMOS_2008_19902', 'COSMOS_2093_20738', 'COSMOS_2142_21230', 'COSMOS_2233_22487', 'COSMOS_2270_23001', 'COSMOS_272_03818', 'COSMOS_272_038181', 'COSMOS_414_05213', 'COSMOS_414_052131', 'COSMOS_507_06120', 'COSMOS_507_061201', 'COSMOS_534_06269', 'COSMOS_534_062691', 'COSMOS_590_06847', 'COSMOS_617_06985', 'COSMOS_626_07005', 'COSMOS_645_07269', 'COSMOS_762_08286', 'COSMOS_789_08591', 'DUMMY_MASS_1_24925', 'ESSA_9_03764', 'ESSA_9_037641', 'METEOR_1-15_06659', 'METEOR_1-15_066591', 'METEOR_1-23_08519', 'METEOR_PRIRODA_12585', 'RADIO_1_11085', 'RADIO_4_13000', 'SURCAL_159_02872', 'SURCAL_159_028721', 'TIMATION_1_02847', 'TIMATION_1_028471', 'Star-61468'

In [4]:
debris_data = pd.read_csv(debris_file,header=None)
is_header = debris_data.iloc[:,0].str.startswith('Time (UTCG)')
group_ids = is_header.cumsum()-1
group_count = group_ids.max() + 1
magnitudes = np.random.normal(4, 0.5, group_count)
headers = debris_data[is_header].iloc[:, 2]
names = headers.str.rsplit('-', n=1, expand=True)[0].str.replace(' ', '')
debris_name_list = names.unique().tolist()

In [ ]:
class SpaceObject():
  def __init__(self, category, name='unknow'):
    self.name = name
    self.category = category # 'sensor', 'debris', 'star'

class Sensor(SpaceObject):
  def __init__(self, name, time, x, y, z, vx, vy, vz, q1, q2, q3, q4, wx, wy, wz, category='sensor'):
    super().__init__(category, name)
    self.time = pd.to_datetime(time)
    self.position = np.array([x, y, z])
    self.velocity = np.array([vx, vy, vz])
    self.quaternions = np.array([q1, q2, q3, q4])
    self.angular_velocity = np.array([wx, wy, wz])
def read_sensor_data(sensor_file):
  sensor_data = pd.read_csv(sensor_file)
  time_list = pd.to_datetime(sensor_data['Time (UTCG)']).to_list()
  sensor=[]
  for i in range(len(sensor_data)):
    name = 'mysensor'
  for sensor_ in sensor_data.iterrows():
    sensor_temp = Sensor(name, sensor_[1]['Time (UTCG)'],sensor_[1]['x (km)'],sensor_[1]['y (km)'],sensor_[1]['z (km)'],sensor_[1]['Velocity x (km/sec)'],sensor_[1]['Velocity y (km/sec)'],sensor_[1]['Velocity z (km/sec)'],sensor_[1]['q1'],sensor_[1]['q2'],sensor_[1]['q3'],sensor_[1]['q4'],sensor_[1]['wx (deg/sec)'],sensor_[1]['wy (deg/sec)'],sensor_[1]['wz (deg/sec)'])
    sensor.append(sensor_temp)
  return sensor, time_list
sensor, time_list = read_sensor_data(sensor_file)

In [11]:
sensor_pos=[]
for i in range(6):
  sensor_pos.append(sensor[i*1000].position)
sensor_pos = np.array(sensor_pos)
washlist = []
for i in range(len(debris_name_list)):
  line = i*6002
  flag = 0
  count=0
  for j in range(6):
    debris_vec=debris_data.iloc[line+1+j*1000,-3:].to_numpy().astype(float)
    if np.linalg.norm(sensor_pos[j]-debris_vec) > 4000:
      count+=1
  if count == 6:
      flag = 1
  if debris_name_list[i] not in access_names:
    flag = 1
  if debris_name_list[i].endswith('1') and debris_name_list[i][:-1] in debris_name_list:
    flag = 1
  if flag == 1:
    washlist.append(list(range(line,line+6002)))
  else:
    print(debris_name_list[i])
washlist = np.array(washlist)
washlist = washlist.flatten().tolist()
reduced_debris_data = debris_data.drop(washlist)
reduced_debris_data.reset_index(drop=True,inplace=True)


0_OPS_5721_09415
ASTRID_1_23464
COBE_20322
COSMOS_1176_11788
COSMOS_1181_11803
COSMOS_1302_12791
COSMOS_1357_13160
COSMOS_1431_13763
COSMOS_1476_14174
COSMOS_1637_15619
COSMOS_1930_18943
COSMOS_2008_19902
COSMOS_2270_23001
COSMOS_507_06120
COSMOS_617_06985
COSMOS_626_07005
COSMOS_645_07269
COSMOS_789_08591
DUMMY_MASS_1_24925
ESSA_9_03764
METEOR_1-15_06659
METEOR_1-23_08519
METEOR_PRIRODA_12585
SURCAL_159_02872
TIMATION_1_02847


In [13]:
print(reduced_debris_data)
reduced_debris_data.to_csv('data/reduced_debris_data.csv',index=False,header=None)

                               0                          1  \
0                    Time (UTCG)  0_OPS_5721_09415 - x (km)   
1       25 Apr 2023 16:00:00.000               -3666.488883   
2       25 Apr 2023 16:00:00.100               -3666.230289   
3       25 Apr 2023 16:00:00.200               -3665.971656   
4       25 Apr 2023 16:00:00.300               -3665.712984   
...                          ...                        ...   
150045  25 Apr 2023 16:09:59.600               -6433.365486   
150046  25 Apr 2023 16:09:59.700               -6433.698670   
150047  25 Apr 2023 16:09:59.800               -6434.031787   
150048  25 Apr 2023 16:09:59.900               -6434.364837   
150049  25 Apr 2023 16:10:00.000               -6434.697821   

                                2                          3  
0       0_OPS_5721_09415 - y (km)  0_OPS_5721_09415 - z (km)  
1                     5487.886700               -2875.777174  
2                     5487.707604               -2876.

In [27]:
star = pd.read_csv(star_file,header=None)
star_magnitude = pd.read_csv(star_magnitude_file,header=None)
star_washlist = []
magnitude_wash_list=[]
for _ in star.iterrows():
  if _[0]%3 == 0:
    name = _[1][1].rsplit('-',maxsplit=1)[0].replace(' ','')
    if name not in access_names:
      star_washlist.append(list(range(_[0],_[0]+3)))
      magnitude_wash_list.append([int(_[0]/3*2),int(_[0]/3*2)+1])
star_washlist = np.array(star_washlist)
star_washlist = star_washlist.flatten().tolist()
magnitude_wash_list = np.array(magnitude_wash_list)
magnitude_wash_list = magnitude_wash_list.flatten().tolist()
print(magnitude_wash_list)
print(len(star_washlist)/3)
print(len(star))
print(len(magnitude_wash_list)/2)
reduced_star = star.drop(star_washlist)
reduced_star.reset_index(drop=True,inplace=True)
reduced_star.to_csv('data/reduced_star.csv',index=False,header=None)
reduced_star_magnitude = star_magnitude.drop(magnitude_wash_list)
reduced_star_magnitude.reset_index(drop=True,inplace=True)
reduced_star_magnitude.to_csv('data/reduced_star_magnitude.csv',index=False,header=None)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,